In [1]:
import pandas as pd
import numpy as np
import string
import pickle
from sklearn.metrics import classification_report 
from nltk.tokenize import RegexpTokenizer
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from gensim.models.doc2vec import Doc2Vec, TaggedDocument 
from gensim.models import Word2Vec 
import nltk
from scipy import spatial

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/azmi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/azmi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/azmi/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /home/azmi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
bugs=pd.read_csv("mozilla_firefox.csv")

In [3]:
bugs.head()

,Issue_id,Priority,Component,Duplicated_issue,Title,Description,Status,Resolution,Version,Created_time,Resolved_time
0,10954,P3,Preferences,NaN,Dialup properties needs to be exposed in prefs,The dialup properties of the profile should be...,RESOLVED,WONTFIX,Trunk,1999-07-30 15:55:51 -0700,2008-05-14 11:44:15 -0700
1,14871,--,General,269442.0,[Find] Find whole word only,Please add Match Whole Word Only option to bro...,RESOLVED,DUPLICATE,Trunk,1999-09-24 14:49:34 -0700,2011-10-05 16:35:31 -0700
2,19118,--,Preferences,NaN,Plug-In Manager (ui for choosing mimetype-plug...,I would really like a plug-in manager for my b...,RESOLVED,WONTFIX,Trunk,1999-11-17 14:58:26 -0800,2013-01-29 11:48:39 -0800
3,54746,P3,Preferences,NaN,Language encodings in font prefs dialog not so...,Language encodings are listed in a seemingly r...,RESOLVED,WORKSFORME,Trunk,2000-09-29 14:12:11 -0700,2013-02-27 15:47:29 -0800
4,56892,P3,General,NaN,Synaptics touchpad scrolling not working,From Bugzilla Helper:; User-Agent: Mozilla/5.0...,RESOLVED,WORKSFORME,unspecified,2000-10-16 14:48:15 -0700,2009-10-14 11:38:29 -0700


In [4]:
bugs=bugs[bugs['Title']!=" "]

In [5]:
bugs['Title_Final'] = bugs['Title'].astype(str)
type(bugs['Title_Final'][4])

str

In [6]:
def remove_punctuation(text):
    no_punct="".join([c for c in text if c not in string.punctuation])
    return no_punct

tokenizer = RegexpTokenizer(r'\w+')

def remove_stopwords(text):
    words=[w for w in text if w not in stopwords.words('english')]
    return words
lemmatizer = WordNetLemmatizer()
def word_lemmatizer(text):
    lem_text = [lemmatizer.lemmatize(i) for i in text]
    return lem_text
stemmer = PorterStemmer()
def word_stemmer(text):
    stem_text = [stemmer.stem(i) for i in text]
    return " ".join(stem_text)
def word_stemmer_list(text):
    stem_text = [stemmer.stem(i) for i in text]
    return stem_text

In [7]:
bugs['Title_Final']= bugs['Title_Final'].apply(lambda x: remove_punctuation(x))
bugs['Title_Final']= bugs['Title_Final'].apply(lambda x: tokenizer.tokenize(x.lower()))
bugs['Title_Final']= bugs['Title_Final'].apply(lambda x: remove_stopwords(x))
bugs['Title_Final']= bugs['Title_Final'].apply(lambda x: word_lemmatizer(x))
bugs['Title_Final_str']= bugs['Title_Final'].apply(lambda x: word_stemmer(x))
bugs['Title_Final_list']= bugs['Title_Final'].apply(lambda x: word_stemmer_list(x))

In [8]:
bugs['Title_Final_list'].head(20)

0                 [dialup, properti, need, expos, pref]
1                             [find, find, whole, word]
2     [plugin, manag, ui, choos, mimetypeplugin, ass...
3            [languag, encod, font, pref, dialog, sort]
4                      [synapt, touchpad, scroll, work]
5        [open, histori, window, select, current, site]
6     [cooki, manag, dont, allow, site, set, remov, ...
7             [today, histori, folder, expand, default]
8           [search, bookmark, access, manag, bookmark]
9                                    [bookmark, option]
10    [fixnot, correctli, retriev, post, data, save,...
11    [show, uri, statu, bar, onmouseov, backforward...
12    [local, problem, bookmark, sort, menu, histori...
13           [ui, allow, extern, handler, intern, type]
14              [mozilla, support, x11, session, manag]
15    [post, result, page, appear, global, histori, ...
16    [nonrespond, window, unc, share, hang, bookmar...
17                                [want, infinit

In [9]:
# Create Skip Gram model 
model = Word2Vec(bugs['Title_Final_list'], min_count = 1,  
                 size = 100, window = 5, sg=1) 

In [10]:
def has_vector_representation(word2vec_model, doc):
    """check if at least one word of the document is in the
    word2vec dictionary"""
    return not all(word not in word2vec_model.vocab for word in doc)

def document_vector(word2vec_model, doc):
    # remove out-of-vocabulary words
    doc = [word for word in doc if word in word2vec_model.vocab]
    return np.mean(word2vec_model[doc], axis=0)

In [31]:
vec_list=[]
for entry in bugs['Title_Final_list']:
    if has_vector_representation(model.wv, entry):
        vec_list.append(document_vector(model.wv, entry))
    else:
        vec_list.append(np.zeros((1,100)))
bugs["vector"] = vec_list


In [44]:
bugs['vector']

0         [0.19635367, -0.40720612, 0.21212259, -0.26033...
1         [0.029688686, -0.3410546, 0.005252581, -0.1907...
2         [0.022631804, -0.47908512, 0.30088642, -0.0549...
3         [-0.09969988, -0.42498899, 0.3972689, -0.46974...
4         [-0.09777178, -0.21430232, 0.1890018, -0.14790...
                                ...                        
115809    [0.0042911144, -0.46271554, 0.23828256, -0.083...
115810    [-0.03614329, -0.3751416, 0.24071841, -0.09389...
115811    [0.13700095, -0.5042338, 0.12149227, 0.0558527...
115812    [0.29436627, -0.3476808, 0.09904309, -0.041370...
115813    [0.050985843, -0.263279, 0.23332639, 0.0947749...
Name: vector, Length: 115798, dtype: object

/home/azmi/Projects/Duplicate Bug Detection/venv/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


TypeError: cannot unpack non-iterable numpy.float32 object

In [36]:
model.wv.similarity("result","icon")

0.55074215